In [4]:
get_ipython().system('pip install requests')
import requests

response=requests.get("https://api.coindesk.com/v1/bpi/currentprice.json")
response.headers.get("Content_Type")                     
json=response.json()
df=pd.json_normalize(json)
df.to_csv("dats.csv")
df
a=list(df.columns[0:5])
a
testcsv=('dats.csv')
testcsv


'dats.csv'

In [5]:
!pip install google-cloud
!pip install google-cloud-bigquery[pandas]
!pip install google-cloud-storage
!pip install google-cloud-bigquery
!pip install pyarrow

zsh:1: no matches found: google-cloud-bigquery[pandas]


In [8]:
from google.cloud import bigquery
#Set environment variables for your notebook
import os 
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/afolakebaiyewu/Downloads/service_key.json'
#Imports google cloud client library and initiates BQ service

cols=df.columns.tolist()
cols[1]
cols


['disclaimer',
 'chartName',
 'time.updated',
 'time.updatedISO',
 'time.updateduk',
 'bpi.USD.code',
 'bpi.USD.symbol',
 'bpi.USD.rate',
 'bpi.USD.description',
 'bpi.USD.rate_float',
 'bpi.GBP.code',
 'bpi.GBP.symbol',
 'bpi.GBP.rate',
 'bpi.GBP.description',
 'bpi.GBP.rate_float',
 'bpi.EUR.code',
 'bpi.EUR.symbol',
 'bpi.EUR.rate',
 'bpi.EUR.description',
 'bpi.EUR.rate_float']

In [43]:
#creating a schema in big query
from google.cloud import bigquery

bigquery_client = bigquery.Client(project="helical-tendril-326615")
dataset_ref = bigquery_client.dataset('gender_names')
cols=df.columns.tolist()

# Prepares a reference to the table
table_ref = dataset_ref.table('dats')
schema =  [
            bigquery.SchemaField(cols[0], 'STRING'),#automating the column names by list
            bigquery.SchemaField(cols[1], 'STRING')
        ]
table = bigquery.Table(table_ref, schema=schema)
table = bigquery_client.create_table(table)
print('table {} created.'.format(table.table_id))

table dats created.


In [44]:
job_config = bigquery.LoadJobConfig(
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,schema =  [
            bigquery.SchemaField(cols[0], 'STRING'),#automating the column names by list
            bigquery.SchemaField(cols[1], 'STRING')
        ],
    autodetect=True
    
)

uri = "gs://test_text_buckets/dats.csv"
load_job = bigquery_client.load_table_from_uri(
    uri,table_ref, job_config=job_config
)  # Make an API request.

load_job.result()  # Waits for the job to complete.

destination_table = bigquery_client.get_table(table)
print("Loaded {} rows.".format(destination_table.num_rows))

Loaded 1 rows.
